In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
digits_data=np.loadtxt("digits.csv",delimiter=",")

In [ ]:
#LDA

In [3]:
def LDA_means(X,target):
        classes=list(np.unique(target))
        mu=np.zeros(shape=(len(classes),X.shape[1]))
        for x in classes:
            mu[x]=np.apply_along_axis(np.mean,0,X[target==x]).reshape(1,X.shape[1])
        return mu

In [4]:
def within_class_variance(X,mus,target):
    classes=list(np.unique(target))
    Sw=np.zeros((mus.shape[1],mus.shape[1]))
    i=1
    for x in classes:
        Swi=np.zeros((mus.shape[1],mus.shape[1]))
        for y in range(0,len(X[target==x])):
            Swi=np.add(Swi,np.subtract(X[target==x][y].reshape(X.shape[1],1),mus[x].reshape(X.shape[1],1)).dot(
                                  np.subtract(X[target==x][y].reshape(1,X.shape[1]),mus[x].reshape(1,X.shape[1]))))
        Sw=np.add(Sw,Swi)
    return Sw

In [5]:
def between_class_variance(X,mus,target):
    classes=list(np.unique(target))
    Sb=np.zeros((mus.shape[1],mus.shape[1]))
    mu=np.mean(X,0)
    for x in classes:
        Sb=np.add(Sb,np.multiply(np.subtract(mus[x],mu).reshape(X.shape[1],1).
                                 dot(np.subtract(mus[x].reshape(1,X.shape[1]),
                                                 mu.reshape(1,X.shape[1]))),len(X[target==x])))
    return Sb

In [6]:
def x_hat_func(Sw,Sb,Xtrain,Xtest):
    final_matrix  = np.dot(np.linalg.pinv(Sw), Sb)
    W,V=np.linalg.eig(final_matrix)
    xhat=Xtrain.dot(V[:,[np.argsort(abs(W))[-1],np.argsort(abs(W))[(W.shape[0]-2)]]].real.reshape(Xtrain.shape[1],2))
    xhat_test=Xtest.dot(V[:,[np.argsort(abs(W))[-1],np.argsort(abs(W))[(W.shape[0]-2)]]].real.reshape(Xtest.shape[1],2))
    return xhat,xhat_test

In [7]:
def prior_prob(target):
    classes=list(np.unique(target))
    prior=[]
    for x in classes:
        prior.append(float(sum(target==x))/float(len(target)))
    return prior

In [8]:
def covar_mean(X,target):
    classes=list(np.unique(target))
    mean=[]
    covar=[]
    for x in classes:
        elts=np.copy(X[target==x])
        mean.append(np.mean(elts,axis=0))
        covar.append(np.cov(np.transpose(elts)))
    return mean,covar

In [9]:
def compute_prob(mean,covar,xtest,target,prior):
    classes=list(np.unique(target.astype(int)))
    prob=np.zeros((xtest.shape[0],1))
    marg_prob=np.zeros((xtest.shape[0],len(classes)))
    for y in classes:
        p=[]
        for z in range(0,xtest.shape[0]):
            class_mean=np.copy(mean[y])
            class_covar=np.copy(covar[y])
            part1=1.0/(2.0*math.pi*math.sqrt(np.linalg.det(class_covar)))
            part2=math.exp(-0.5*np.dot((xtest[z]-class_mean),np.dot(np.linalg.inv(class_covar),np.transpose(xtest[z]-class_mean))))
            p.append( part2 * part1 *prior[y])
        marg_prob[:,y]=p
        #break
    return marg_prob

In [10]:
def accuracy(ytest,yhat):
    right=float((yhat==ytest).sum())
    accuracy=right/float(ytest.shape[0])
    return accuracy

In [11]:
def cross_validation(X):
    np.random.shuffle(X)
    index=[]
    for i in range(0,10):
        index.append(range(X.shape[0]*i/10,X.shape[0]*(i+1)/10))
    accura=[]
    accura_train=[]
    for i in range(0,len(index)):
        Xtest=np.copy(X[index[i],:-1])
        ytest=np.copy(X[index[i],-1])
        Xtrain=np.delete(X[:,:-1],index[i],axis=0)
        ytrain=np.delete(X[:,-1],index[i],axis=0)
        mus=LDA_means(Xtrain,ytrain)
        Sw=within_class_variance(Xtrain,mus,ytrain)
        Sb=between_class_variance(Xtrain,mus,ytrain)
        xhat,xhat_test=x_hat_func(Sw,Sb,Xtrain,Xtest)
        prior=prior_prob(ytrain)
        mean,covar=covar_mean(xhat,ytrain)
        marg_prob=compute_prob(mean,covar,xhat_test,ytrain,prior)
        marg_prob_train=compute_prob(mean,covar,xhat,ytrain,prior)
        yhat=np.argsort(marg_prob,axis=1)[:,-1]
        yhat_train=np.argsort(marg_prob_train,axis=1)[:,-1]
        accura.append(accuracy(yhat.astype(int),ytest))
        accura_train.append(accuracy(yhat_train.astype(int),ytrain))
    return accura,accura_train

In [12]:
acc,acc_train=cross_validation(digits_data)
error=list(1.0-np.array(acc))
error_train=list(1.0-np.array(acc_train))
print "Traing Error"
print error_train
print "------------"
print "Standard Deveation Train"
print np.std(error_train)
print "------------"
print "Train error mean"
print np.mean(error_train)
print "------------"
print "Test Error"
print error
print "------------"
print "Test error mean"
print np.mean(error)
print "------------"
print "Standard Deveation test"
print np.std(error)

(64L, 64L)
(64L, 64L)
(64L, 64L)
(64L, 64L)
(64L, 64L)
(64L, 64L)
(64L, 64L)
(64L, 64L)
(64L, 64L)
(64L, 64L)
Traing Error
[0.29728059332509271, 0.32467532467532467, 0.30550401978973407, 0.28862793572311496, 0.32282003710575136, 0.24922696351267781, 0.28862793572311496, 0.29746444032158315, 0.28200371057513918, 0.27272727272727271]
------------
Standard Deveation Train
0.021385200405
------------
Train error mean
0.292895823348
------------
Test Error
[0.32402234636871508, 0.37777777777777777, 0.34999999999999998, 0.27932960893854752, 0.36111111111111116, 0.30555555555555558, 0.33519553072625696, 0.37222222222222223, 0.31666666666666665, 0.33333333333333337]
------------
Test error mean
0.33552141527
------------
Standard Deveation test
0.0292320153827
